<img width="500" style="float:center" 
     src="https://images6.alphacoders.com/119/1191374.jpg" />

# Twitter User Analysis on #SquidGame Using Spark

# Sections
* [Description](#0)
* [1. Setup](#1)
  * [1.1 Start Hadoop](#1.1)  
  * [1.2 PySpark environment setup](#1.2)
  * [1.3 Create SparkSession](#1.3)
* [2. Schema and DataFrames](#2)
  * [2.1 Creating the DataFrames](#2.1)
      * [2.a) Streaming DataFrame](#2.a)
      * [2.b) Batch DataFrame](#2.b)
  * [2.2 Join: Union to Connect both Data Frames](#2.2)
  * [2.3 Adding three new columns to our Data-Frame](#2.3)
* [3. Analyzing our Data](#3)
* [4. TearDown](#4)
  * [3.1 Stop Hadoop](#4.1)

<a id='0'></a>
## Description
<p>
<div> To find insights in regards to the conversation of Squid Game on Twitter, the exploratory analysis will be guided by the central premise of understanding the users and based on the following four different pillars: who are the users? Where are the users?  How are they tweeting? and What are they saying?
 </div>
 
 Sub-queries within the three main pillars:

<ul class="roman">
 <li>Who are the users?</li>
    <ul class="square">
  <li>What is the ratio of Verified accounts in the conversation of SquidGame?</li>
  <li> Who are the Top 5 users with more tweets published about #SquidGame?</li>
  <li> Who are the top 5 verified accounts with the highest following?</li>
  <li> Who are the Top 5 most followed accounts?</li>
  <li> Does size matter? Does being verify guarantee following? </li>
 </ul>
 <li>What are users saying?
 <ul class="square">
  <li>What are the tweets of the verified accounts with the highest following?</li>
  <li> What are the top 5 hashtags?</li>
  <li> What are the most used words in the conversation?</li>
 </ul>
 </li>
 <li>How are they tweeting?
     <ul class="square">
  <li> What are the 5 top locations users were tweeting at? </li>
  <li> What are the top 5 operating systems used? </li>
  <li> Do users tweet more with their phones or using their computers? </li>
  <li> Does typing status matter per operating system? </li>
  <li> What are the top 10 languages written per tweet distribution?</li>
  <li> What is the average of words per tweet using the #SquidGame?</li>
  <li> What are the Max and Average Number of Hashtags per tweet?</li>
  <li> What is the ratio of typer status? </li>
  <li> Were tweets retweeted?</li>
  <li> How much are users tweeting by date? Can you find a pattern? </li>
  <li> What are the top 5 hashtags?</li>
  <li> Is there a pattern of type status by date?</li>
  <li> Is there a pattern between verified accounts and the date they were typed?</li>
     
</ul>
</ul>    
</p>


<a id='1'></a>
## 1. Setup

Since we are going to process data stored from HDFS, we had to start the service. Furthemore, we also need to set up the PySpark enviroment & create a SparkSession. 

<a id='1.1'></a>
### 1.1 Start Hadoop

Please, in order to verify this notebook, start HDFS in your computer. ☺️

```sh
hadoop-start.sh
```

<a id='1.2'></a>
### 1.2 PySpark environment setup 🚀

In [2]:
import findspark
findspark.init()

from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession

sc = SparkContext.getOrCreate()
spark = SparkSession(sc)

import pandas as pd
pd.set_option('display.max_colwidth', None)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


<a id='1.3'></a>
### 1.3 Create SparkSession 💻

In [3]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars /opt/hive3/lib/hive-hcatalog-core-3.1.2.jar pyspark-shell'

In [4]:
from pyspark.sql.session import SparkSession

spark = (SparkSession.builder
    .appName("Squid Game - Group G")
    .config("spark.sql.warehouse.dir","hdfs://localhost:9000/warehouse")
    .config("spark.sql.legacy.timeParserPolicy","LEGACY")
    .enableHiveSupport()
    .getOrCreate())

<a id='2'></a>
## 2. Schema and DataFrames 🖼
We will be analyzing tweets about Squid Game on Twitter for our analysis. Importantly, we will use both batch data sources with a dataset retrieved from Kaggle and streaming data sources retrieved from Twitter using  Tweepy, an API, to extract tweets ingested with Nifi and then stored in HDFS. Hence, for our analysis, we had to create schemas and data frames with the same column names, types, & numbers to combine both using Union to one master dataframe.

<a id='2.1'></a>
### 2.1 Creating the DataFrames

<a id='2.a'></a>
#### a) Streaming DataFrame:

In [5]:
streaming_general_DF = (spark.read
              .json("hdfs://localhost:9000/user/osbdet/squidgame"))

In [6]:
from pyspark.sql.functions import *
streaming_DF_normalized = streaming_general_DF.select(col("user.name").alias("name"), col("user.location").alias("location"), col("user.description").alias("description"), col("user.created_at").alias("twitter_created_at"), col("user.followers_count").alias("followers_count"), col("user.friends_count").alias("friends_count"), col("user.favourites_count").alias("favourites_count"), col("user.verified").alias("verified"), "created_at", "text", "source", "retweeted")
streaming_DF_normalized.printSchema()

root
 |-- name: string (nullable = true)
 |-- location: string (nullable = true)
 |-- description: string (nullable = true)
 |-- twitter_created_at: string (nullable = true)
 |-- followers_count: long (nullable = true)
 |-- friends_count: long (nullable = true)
 |-- favourites_count: long (nullable = true)
 |-- verified: boolean (nullable = true)
 |-- created_at: string (nullable = true)
 |-- text: string (nullable = true)
 |-- source: string (nullable = true)
 |-- retweeted: boolean (nullable = true)



In [7]:
streaming_DF_normalized.select('created_at').show(1, False) #Date in streaming data is "EEE MMM dd HH:mm:ss Z yyyy"

+------------------------------+
|created_at                    |
+------------------------------+
|Thu Dec 02 10:04:28 +0000 2021|
+------------------------------+
only showing top 1 row



#### We found out that the date format, apart of being a string, is also as "EEE MMM dd HH:mm:ss Z yyyy", which is different than in the batch dataframe. Hence, there was a need to covert them to the same date format, and also transform the StringTpe to a DateType.

In [8]:
from pyspark.sql.functions import unix_timestamp, from_unixtime

streaming_DF_normalized = streaming_DF_normalized.withColumn('created_at_date', from_unixtime(
    unix_timestamp("created_at", "EEE MMM dd HH:mm:ss Z yyyy")))

streaming_DF_normalized = streaming_DF_normalized.withColumn('created_at_date', \
                               to_date('created_at_date', 'yyyy-MM-dd'))

streaming_DF_normalized.printSchema() # new columns with datetype successfully created
streaming_DF_normalized.select('created_at_date').show(1) #date works 

root
 |-- name: string (nullable = true)
 |-- location: string (nullable = true)
 |-- description: string (nullable = true)
 |-- twitter_created_at: string (nullable = true)
 |-- followers_count: long (nullable = true)
 |-- friends_count: long (nullable = true)
 |-- favourites_count: long (nullable = true)
 |-- verified: boolean (nullable = true)
 |-- created_at: string (nullable = true)
 |-- text: string (nullable = true)
 |-- source: string (nullable = true)
 |-- retweeted: boolean (nullable = true)
 |-- created_at_date: date (nullable = true)

+---------------+
|created_at_date|
+---------------+
|     2021-12-02|
+---------------+
only showing top 1 row



In [9]:
streaming_DF_normalized.count()

5636

<a id='2.b'></a>
#### b) Batch DataFrame:

In [10]:
from pyspark.sql.types import *
batch_customschema = StructType([
StructField(
 "name", StringType(), True),StructField(
 "location", StringType(), True),StructField(
 "description", StringType(), True),StructField(
 "twitter_created_at", StringType(), True),StructField(
 "followers_count", LongType(), True),StructField(
 "friends_count", LongType(), True),StructField(
 "favourites_count", LongType(), True),StructField(
 "verified", BooleanType(), True),StructField(
 "created_at", StringType(), True),StructField(
 "text", StringType(), True),StructField(
 "source", StringType(), True),StructField(
 "retweeted", BooleanType(), True)])

In [11]:
batch_DF = spark.read.format("csv").options(header="true", multiline="true")\
    .schema(batch_customschema).load("hdfs://localhost:9000/user/osbdet/squidgame_batch")

batch_DF.printSchema()
batch_DF.select('created_at').show(1, False)

root
 |-- name: string (nullable = true)
 |-- location: string (nullable = true)
 |-- description: string (nullable = true)
 |-- twitter_created_at: string (nullable = true)
 |-- followers_count: long (nullable = true)
 |-- friends_count: long (nullable = true)
 |-- favourites_count: long (nullable = true)
 |-- verified: boolean (nullable = true)
 |-- created_at: string (nullable = true)
 |-- text: string (nullable = true)
 |-- source: string (nullable = true)
 |-- retweeted: boolean (nullable = true)

+-------------------------+
|created_at               |
+-------------------------+
|2021-10-06 12:05:38+00:00|
+-------------------------+
only showing top 1 row



In [12]:
from pyspark.sql.types import DateType
batch_DF = batch_DF.withColumn("created_at_date", batch_DF['created_at'].cast(DateType()))
batch_DF.printSchema()

root
 |-- name: string (nullable = true)
 |-- location: string (nullable = true)
 |-- description: string (nullable = true)
 |-- twitter_created_at: string (nullable = true)
 |-- followers_count: long (nullable = true)
 |-- friends_count: long (nullable = true)
 |-- favourites_count: long (nullable = true)
 |-- verified: boolean (nullable = true)
 |-- created_at: string (nullable = true)
 |-- text: string (nullable = true)
 |-- source: string (nullable = true)
 |-- retweeted: boolean (nullable = true)
 |-- created_at_date: date (nullable = true)



In [13]:
batch_DF.select("created_at_date").show(1) #date works 

+---------------+
|created_at_date|
+---------------+
|     2021-10-06|
+---------------+
only showing top 1 row



In [14]:
batch_DF.count()

80793

<a id='2.2'></a>
### 2.2 Join: Union to Connect both DataFrames 🫂
To connect both datasets, streaming and not streaming. Count() was called to make sure the union worked and the sum between batch and stream worked correctly.


In [15]:
union_DF = streaming_DF_normalized.union(batch_DF)
union_DF.count()

86429

<a id='2.3'></a>
### 2.3 Adding three new columns to our Data-Frame

In [16]:
import pyspark.sql.functions as f

#### Adding a new column with the word count: 🧮


In [17]:
union_DF = union_DF.withColumn('wordCount', f.size(f.split(f.col('text'), ' ')))
union_DF.printSchema() # used to check if the column was correctly added 

root
 |-- name: string (nullable = true)
 |-- location: string (nullable = true)
 |-- description: string (nullable = true)
 |-- twitter_created_at: string (nullable = true)
 |-- followers_count: long (nullable = true)
 |-- friends_count: long (nullable = true)
 |-- favourites_count: long (nullable = true)
 |-- verified: boolean (nullable = true)
 |-- created_at: string (nullable = true)
 |-- text: string (nullable = true)
 |-- source: string (nullable = true)
 |-- retweeted: boolean (nullable = true)
 |-- created_at_date: date (nullable = true)
 |-- wordCount: integer (nullable = false)



#### Using describe to understand the dispersion of words by tweet, in order to make a boolean dataframe as shown below:


In [18]:
words_stats = union_DF.describe(["wordCount"])
words_stats.toPandas()

,summary,wordCount
0,count,86429
1,mean,13.966631570421965
2,stddev,6.203672905729246
3,min,-1
4,max,93


#### Creating a boolean column to categorize users in how much they type: 🤔

In [19]:
union_DF = union_DF\
    .withColumn("TyperStatus", when(col("wordCount") <=0, "no_text")\
                        .when((col("wordCount")> 0) & (col("wordCount")<=7.76), "short_typer")\
                        .when((col("wordCount")> 7.76) & (col("wordCount")<=20.17), "medium_typer")\
                        .when((col("wordCount")> 20.17) & (col("wordCount")<=220), "high_typer"))\

union_DF.select("wordCount", "TyperStatus").distinct().show(3) #called to check if the content was correctly added
union_DF.printSchema() #called to check if the column was correctly added

+---------+------------+
|wordCount| TyperStatus|
+---------+------------+
|        5| short_typer|
|       51|  high_typer|
|       10|medium_typer|
+---------+------------+
only showing top 3 rows

root
 |-- name: string (nullable = true)
 |-- location: string (nullable = true)
 |-- description: string (nullable = true)
 |-- twitter_created_at: string (nullable = true)
 |-- followers_count: long (nullable = true)
 |-- friends_count: long (nullable = true)
 |-- favourites_count: long (nullable = true)
 |-- verified: boolean (nullable = true)
 |-- created_at: string (nullable = true)
 |-- text: string (nullable = true)
 |-- source: string (nullable = true)
 |-- retweeted: boolean (nullable = true)
 |-- created_at_date: date (nullable = true)
 |-- wordCount: integer (nullable = false)
 |-- TyperStatus: string (nullable = true)



#### Bringing colour to the main data frame. Adding emojis by their typing status: 🎊

In [20]:
union_DF = union_DF.withColumn("emoji",
                    f.when(union_DF["TyperStatus"]=="short_typer",f.lit("🙅🏼‍♀️"))\
                      .when(union_DF["TyperStatus"]=="medium_typer",f.lit("😁"))\
                      .when(union_DF["TyperStatus"]=="high_typer",f.lit("👑"))\
                      .otherwise(f.lit(None)))
union_DF.select("wordCount", "TyperStatus", "emoji").distinct().show(3) #called to check if the content was correctly added
union_DF.printSchema() #called to check if the column was correctly added

+---------+------------+-------+
|wordCount| TyperStatus|  emoji|
+---------+------------+-------+
|        7| short_typer|🙅🏼‍♀️|
|       58|  high_typer|     👑|
|        8|medium_typer|     😁|
+---------+------------+-------+
only showing top 3 rows

root
 |-- name: string (nullable = true)
 |-- location: string (nullable = true)
 |-- description: string (nullable = true)
 |-- twitter_created_at: string (nullable = true)
 |-- followers_count: long (nullable = true)
 |-- friends_count: long (nullable = true)
 |-- favourites_count: long (nullable = true)
 |-- verified: boolean (nullable = true)
 |-- created_at: string (nullable = true)
 |-- text: string (nullable = true)
 |-- source: string (nullable = true)
 |-- retweeted: boolean (nullable = true)
 |-- created_at_date: date (nullable = true)
 |-- wordCount: integer (nullable = false)
 |-- TyperStatus: string (nullable = true)
 |-- emoji: string (nullable = true)



<a id='3'></a>
## 3. Analyzing our Data 🕵🏼‍♀️


In [21]:
import pandas as pd

#### 1.Who are the users? Ratio of Verified accounts in the conversation of SquidGame:

In [22]:
verified_ratio_df = union_DF.groupBy("verified").agg(count("*").alias("Count")).sort(col("Count").desc())

verified_ratio_df.toPandas()

# 4% of the users tweeting about SquidGame have verified accounts
# 95% have unverified accounts

,verified,Count
0,False,82026
1,True,3313
2,None,1090


#### 1. Who are the users? Top 5 common users with more tweets published about SquidGame:

In [23]:
top_users_df = (union_DF.groupBy("name")
          .agg(count("*").alias("Count"))
          .orderBy(desc("Count"))
          .limit(5))
top_users_df.toPandas()

,name,Count
0,Sale X Low,478
1,John Doge,263
2,7-Ainu,170
3,Ofofonobs,147
4,havingalaff,110


#### 1.Who are the users? Verified distinct accounts that talked about SquidGame with highest following:

In [24]:
verified_most_followed = union_DF.where(union_DF["verified"] == "True").select("Name").distinct().sort(union_DF["followers_count"].desc())

verified_most_followed.toPandas().head()

,Name
0,9GAG
1,Vogue Magazine
2,billboard
3,E! News
4,Twitter TV


#### 1.Who are the users?  - Top 5 most followed users in our SquidGame Conversation: 

In [27]:
most_followed_df = (union_DF
          .groupBy("name")
          .agg(max("followers_count").alias("followers_count"))
          .orderBy(desc("followers_count"))
          .limit(5))
most_followed_df.toPandas()

# They ended up being the same as the verified! 

,name,followers_count
0,9GAG,16846417
1,Vogue Magazine,14359408
2,billboard,12588047
3,E! News,11710347
4,Twitter TV,9207638


#### 1.Who are the users? Does size matter? Follower Count with Verification status:


In [28]:
size_summary = batch_DF.groupBy("verified")\
        .agg(
            f.count("*").alias("Count"),
            f.max("followers_count").alias("Max_Followers_Count"),
            f.round(f.avg("followers_count")).alias("Avg_followers_count"),
            f.min("followers_count").alias("Min_Followers_Count"))\
        .sort(f.col("Count").desc())\
        .show(10)

size_summary

# Here we can see that one of the verified accounts actually has only 14 followers, showing that being verified does not mean large following.

+--------+-----+-------------------+-------------------+-------------------+
|verified|Count|Max_Followers_Count|Avg_followers_count|Min_Followers_Count|
+--------+-----+-------------------+-------------------+-------------------+
|   false|76411|             944282|             2226.0|                  0|
|    true| 3292|           16846417|           383938.0|                 14|
|    null| 1090|              52912|             8494.0|                  1|
+--------+-----+-------------------+-------------------+-------------------+



#### 1.Who are the users? Popularity of Twitter users on the Top 10 locations:

In [29]:
union_DF.groupBy("location")\
        .agg(
            f.count("*").alias("Count"),
            f.max("followers_count").alias("Max_Followers_Count"),
            f.round(f.avg("followers_count")).alias("Avg_followers_count"),
            f.min("followers_count").alias("Min_Followers_Count"))\
        .sort(f.col("Avg_followers_count").desc())\
        .show(10)

# Lakc of data preparation does not allow us to draw a conclusion for this insigths since count should be proportional. 

+--------------------+-----+-------------------+-------------------+-------------------+
|            location|Count|Max_Followers_Count|Avg_followers_count|Min_Followers_Count|
+--------------------+-----+-------------------+-------------------+-------------------+
|           Bollywood|    3|            4607387|          4607315.0|            4607201|
|  Back in Studio 6B!|   14|            4441295|          4437418.0|            4436402|
|            Universe|   13|           16846417|          4211646.0|                  7|
|           Studio 1A|    2|            4200203|          4200052.0|            4199900|
|    Karachi,Pakistan|    4|            3699938|          3658578.0|            3640328|
|South Kensington,...|    1|            2320525|          2320525.0|            2320525|
|      Beijing, China|    6|            4268614|          2173817.0|               1211|
|        In the Arena|    1|            2135604|          2135604.0|            2135604|
|     DGP HQs LUCKNOW

#### 2. What are users tweeting about? Tweets of the verified accounts with highest following were:

In [30]:
verified_most_followed_tweets = union_DF\
.where(union_DF["verified"] == "True")\
.select("Name", "text")\
.sort(union_DF["followers_count"].desc())

verified_most_followed_tweets.toPandas().head(10)

,Name,text
0,9GAG,Auntie wondering why her snack is so popular these days #SquidGame\n\n📹 shinyong412 | IG https://t.co/DISLkVDwUq
1,9GAG,[🔉] The clock that we didn't know we need #SquidGame\n\n📹 gaspar.3d (@3dgaspar) &amp; nanvo | IG https://t.co/oHqvu7uOaS
2,9GAG,He hasn't been able to go home for a week. #SquidGame\nhttps://t.co/CYj2kfYh1K
3,Vogue Magazine,#SquidGame is worth the hype https://t.co/uKzgO87xGm
4,billboard,"Named after the hit Netflix series, Soulja Boy raps over an instrumental of the #SquidGame theme song on his latest… https://t.co/ZUcRGccQY2"
5,billboard,"Meek Mill has been paying close attention to Netflix's viral new series #SquidGame, and the rapper thinks he spotte… https://t.co/HMe8tTQ9GY"
6,E! News,What a prrrfect excuse to rewatch #SquidGame. (🎥: @sayanamusic/TikTok) https://t.co/g6NdwPgJV1
7,Twitter TV,when Gi-hun and Sang-woo face off in #SquidGame \n\nhttps://t.co/2S14g5bRhU
8,Louis Vuitton,#LouisVuitton is pleased to welcome #HoYeonJung. The Netflix #SquidGame star joins as new Global House Ambassador f… https://t.co/UOoBt8YTpD
9,Hindustan Times,"""#Squid was launched as a """"play-to-earn"""" cryptocurrency for #SquidGame project - an online tournament that will be l… https://t.co/C578BkJs4t"""


#### 2. What are users tweeting about? Top 5 hashtags:

In [32]:
df = (streaming_general_DF
      .select(explode("entities.hashtags.text").alias("hashtag"))
      .groupBy("hashtag")
      .agg(count("*").alias("total"))
      .orderBy(desc("total"))
      .limit(5))
      
df.toPandas()

# 60% of the Top 5 hashtags are related to the intersection of K-Pop and SquidGame

,hashtag,total
0,SquidGame,3224
1,BTS,972
2,squidgame,442
3,PTD_ON_STAGE_LA,222
4,btsfanart,143


#### 2. What are users tweeting about? What are the most used words in the conversation?

In [79]:
union_DF.withColumn('word', f.explode(f.split(f.col('text'), ' '))) \
  .groupBy('word') \
  .count() \
  .sort('count', ascending=False) \
  .show(10)

# This dataset would need to be furthered cleaned (i.e: drop of buzzwords) for a proper insight to be drawn. 

+----------+-----+
|      word|count|
+----------+-----+
|#SquidGame|43648|
|       the|31106|
|        to|18227|
|         I|17930|
|         a|16172|
|        is|15493|
|       and|15459|
|        of|15012|
|        in|12219|
|          |11197|
+----------+-----+
only showing top 10 rows



#### 3. How are they tweeting? Top 5 Locations:

In [33]:
top_locations_df = (union_DF.groupBy("location")
          .agg(count("*").alias("Count"))
          .orderBy(desc("Count"))
          .limit(5))
top_locations_df.toPandas()

,location,Count
0,None,26436
1,"Los Angeles, CA",911
2,"London, England",723
3,USA,716
4,United States,711


Since the information in location does not follow a specific format, many rows represent the same values. Hence, we had to increase the show to 20, and make the manual calculations:
The top regions are: United States, United Kingdoom, India, Canada & United Arab Emirates.

#### 3. How are they tweeting? Top 5 sources used by Twitter Users:

In [81]:
sources_ratio_df = (union_DF.groupBy("source")
          .agg(count("*").alias("Count"))
          .orderBy(desc("Count"))
          .limit(5))
sources_ratio_df.toPandas()

,source,Count
0,Twitter for iPhone,31384
1,Twitter for Android,24084
2,Twitter Web App,15972
3,"<a href=""http://twitter.com/download/android"" rel=""nofollow"">Twitter for Android</a>",2756
4,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",1564


#### 3. How are they tweeting? Does typing status matter per operating system?

In [135]:
status_by_op_system = (union_DF\
      .select('source', "TyperStatus"))\
      .groupBy('source', "TyperStatus")\
      .agg(count("*").alias("count"))\
      .orderBy(desc('count'))\
    .limit(10)

status_by_op_system.toPandas()


# Users that use Iphones tweet more than other uses with other systems.

,source,TyperStatus,count
0,Twitter for iPhone,medium_typer,19674
1,Twitter for Android,medium_typer,15834
2,Twitter Web App,medium_typer,11033
3,Twitter for iPhone,high_typer,5908
4,Twitter for iPhone,short_typer,5802
5,Twitter for Android,high_typer,4245
6,Twitter for Android,short_typer,4005
7,Twitter Web App,high_typer,2958
8,Twitter Web App,short_typer,1981
9,"<a href=""http://twitter.com/download/android"" rel=""nofollow"">Twitter for Android</a>",medium_typer,1865


#### 3. How are they tweeting? What is the language most used in the conversation?:

In [34]:
streaming_general_DF.groupBy("lang")\
       .agg(
        count("*").alias("Count"))\
        .sort(col("Count").desc())\
        .show(1)

#English is the most used language! 

+----+-----+
|lang|Count|
+----+-----+
|  en| 3658|
+----+-----+
only showing top 1 row



#### 3. How are they tweeting? Average of words per tweet using the #SquidGame:

In [35]:
union_DF.select(avg(size(split("text", " "))).alias("Average of Words per Tweet")).toPandas()

,Average of Words per Tweet
0,13.966632


#### 3. How are they tweeting? What are the Max and Average Number of Hashtags per tweet?

In [36]:
streaming_general_DF.select(
            max(size("entities.hashtags")).alias("max # per tweet"),
            avg(size("entities.hashtags")).alias("average # per tweet")
).toPandas()

,max # per tweet,average # per tweet
0,23,2.293116


#### 3. How are they tweeting? Ratio of typer status: 

In [37]:
emoji_ratio_df = (union_DF.groupBy("emoji",lower("TyperStatus").alias("TyperStatus"))
          .agg(count("*").alias("Count"))
          .orderBy(desc("Count"))
          .limit(5))
emoji_ratio_df.toPandas()

#65% of users are medium typers! 😁

,emoji,TyperStatus,Count
0,😁,medium_typer,56551
1,👑,high_typer,14962
2,🙅🏼‍♀️,short_typer,14137
3,None,no_text,779


#### 3. How are they tweeting?  Chain ideas: Were tweets retweeted?

In [38]:
sources_ratio_df = (union_DF.groupBy("retweeted")
          .agg(count("*").alias("Count"))
          .orderBy(desc("Count"))
          .limit(5))
sources_ratio_df.toPandas()

#No tweets were retweeted!

,retweeted,Count
0,False,84250
1,None,2179


#### 3. How are they tweeting? Checking how many had their geotag enabled:

In [39]:
streaming_general_DF.where("user.geo_enabled=true").select("user.id").distinct().count()

#1081 accounts had their geo tag enabled.

1081

#### 3. How are they tweeting? How much are users tweeting by date? Can you find a pattern?

In [87]:
date_count_df = (union_DF.groupBy("created_at_date")
          .agg(count("*").alias("Count"))
          .orderBy(asc("created_at_date")))

date_count_df.toPandas().to_csv('date_count_df.csv') #to download dates and tweet count as a .csv file for graph ploting

#After analyzing the CSV document and showed in the presentation, we found that the conversation is indeed declining since its release date.

#### 3. How are they tweeting? Is there a pattern between typer_status and the date in which tweets were tweeted?

In [119]:
typer_status_by_date = (union_DF\
      .select('created_at_date', "TyperStatus"))\
      .groupBy('created_at_date', "TyperStatus")\
      .agg(count("*").alias("count"))\
      .orderBy(asc('created_at_date'))

typer_status_by_date = typer_status_by_date.toPandas()
typer_status_by_date

typer_status_by_date.to_csv('typer_status_by_date') #to create csv document to be analyzed]

#After analyzing the CSV file, no pattern was found!

#### 3. How are they tweeting? Is there a pattern between verified accounts and when they type?

In [112]:
verified_by_date = (union_DF\
      .select('created_at_date', "verified"))\
      .groupBy('created_at_date', "verified")\
      .agg(count("*").alias("count"))\
      .orderBy(asc('created_at_date'))\
    .filter(union_DF.verified == True)

verified_by_date = verified_by_date.toPandas()
verified_by_date
verified_by_date.to_csv('verified_by_date')

#After analyzing the CSV, when have found that as the hype decreases, so does the tweets of verified accounts. 

In [114]:
union_DF.filter(union_DF.verified == True).count() # to find how many tweets by verified accounts

3313

#### To understand the integers in our dataframe:

In [89]:
union_stats = union_DF.describe(["followers_count", "friends_count", "favourites_count"])
union_stats.toPandas()

,summary,followers_count,friends_count,favourites_count
0,count,85436,85464,85523
1,mean,17067.772917739596,1058.3278456426096,18480.792278100627
2,stddev,238012.74928652175,6556.1210423413995,48883.9275267829
3,min,0,0,0
4,max,16846417,1211576,1144792


<a id='4'></a>
## 4. Tear Down

Stop Hadoop

Open a terminal and execute
```sh
hadoop-stop.sh
```